In [18]:
import pandas as pd
import numpy as np
import json
from collections import defaultdict

In [34]:
results:dict = json.load(open("/media/stuff/Pavel/Code/csv_detective_api/csv_data_rules.json"))

In [35]:
results["344a19ea-0eec-46e8-aaeb-6f8d53ade88b"]

{'encoding': 'UTF-8',
 'separator': ',',
 'header_row_idx': 0,
 'header': ['identifiant',
  'type_de_structure',
  'libelle_type_de_structure',
  'libelle_court',
  'libelle_long',
  'siege_administratif',
  'geolocalisation',
  'latitude',
  'longitude',
  'adresse',
  'complement_adresse',
  'code_postal',
  'localite',
  'site_internet',
  'compte_twitter',
  'siret',
  'element_wikidata',
  'compte_facebook',
  'compte_linkedin',
  'compte_viadeo',
  'compte_dailymotion',
  'compte_youtube',
  'code_commune',
  'commune',
  'code_unite_urbaine',
  'unite_urbaine',
  'code_departement',
  'departement',
  'code_academie',
  'academie',
  'code_region',
  'region'],
 'total_lines': 18,
 'heading_columns': 0,
 'trailing_columns': 0,
 'ints_as_floats': [],
 'columns_rb': {'siege_administratif': ['booleen'],
  'geolocalisation': ['latlon_wgs'],
  'latitude': ['latitude_wgs',
   'longitude_wgs',
   'longitude_wgs_fr_metropole',
   'longitude_l93'],
  'longitude': ['latitude_wgs',
   'lon

In [36]:
# Find the distinct types found by the ML method

all_types = set([])

for id_, metadata in results.items():
    if "columns_rb" in metadata and metadata["columns_rb"]:
        for col_name, type_ in metadata["columns_rb"].items():
            all_types.update(type_)
        
"""
{'adresse',
 'booleen',
 'code_commune_insee',
 'code_csp_insee',
 'code_departement',
 'code_fantoir',
 'code_postal',
 'code_region',
 'commune',
 'csp_insee',
 'date',
 'date_fr',
 'datetime_iso',
 'departement',
 'email',
 'insee_ape700',
 'insee_canton',
 'iso_country_code',
 'jour_de_la_semaine',
 'json_geojson',
 'latitude_l93',
 'latitude_wgs',
 'latitude_wgs_fr_metropole',
 'latlon_wgs',
 'longitude_l93',
 'longitude_wgs',
 'longitude_wgs_fr_metropole',
 'pays',
 'region',
 'sexe',
 'siren',
 'siret',
 'tel_fr',
 'twitter',
 'uai',
 'url',
 'year'}
 """

"\n{'adresse',\n 'code_commune_insee',\n 'code_departement',\n 'code_postal',\n 'code_region',\n 'commune',\n 'date',\n 'datetime_iso',\n 'departement',\n 'json_geojson',\n 'latitude_l93',\n 'latitude_wgs',\n 'latlon_wgs',\n 'longitude_l93',\n 'longitude_wgs',\n 'pays',\n 'region',\n 'sexe',\n 'siren',\n 'siret',\n 'url',\n 'year'}\n "

In [43]:
# Types not to annotate (by me)
non_annotate_types = {
 'code_csp_insee',
 'code_fantoir',
 'csp_insee',
 'insee_ape700',
 'insee_canton',
 'latitude_l93',
 'latitude_wgs',
 'latitude_wgs_fr_metropole',
 'latlon_wgs',
 'longitude_l93',
 'longitude_wgs',
 'longitude_wgs_fr_metropole',
 'uai',
}



In [41]:
# Create dict with iinverted index {type: (col_name, id_)}
iindex_types_cols = defaultdict(list)
for id_, metadata in results.items():
    if "columns_rb" in metadata and metadata["columns_rb"]:
        for col_name, type_ in metadata["columns_rb"].items():
            iindex_types_cols[type_[0]].append((col_name, id_))

In [45]:
def get_sample(results, id_, header, n_sample=10):
    encoding = "utf-8"
    separator = ","
    if id_ in results:
        if "encoding" in results[id_]:
            encoding = results[id_]["encoding"]
        if "separator" in results[id_]:
            separator = results[id_]["separator"]
    try:        
        df = pd.read_csv("/data/datagouv/datagouv_full", sep=separator, encoding=encoding)
    except:
        print(f"Could not read {id_} file")
        return []
    
    if header in df.columns:
        sampled_values = df[header].dropna().sample(n=n_sample, random_state=42).values
        return sampled_values
    else:
        print(f"Could not find column {header} in file {id_}")
        return []
    
    
    
    
            
        

In [42]:
iindex_types_cols["siren"]

[('EPCI 2016', 'c7d9fa8f-fbea-40d7-8090-e6226910439c'),
 ('FINESS géographique', '0bf69358-fca8-496b-961e-1dc2faeba89c'),
 ('finess', '3a1987ed-a4f8-4da9-a167-4ae01272a768'),
 ('finessEJ', '3a1987ed-a4f8-4da9-a167-4ae01272a768'),
 ('Code EPCI', '69e3f577-0384-4488-9305-5f05534b0dd1'),
 ('siren', '5bbd76e7-6966-47c4-9f32-9a678b963bdc'),
 ('finess', '0e5bb417-18b1-48d4-886c-f796b6b39f6b'),
 ('ref:FR:FINESS', 'db19d4e9-779e-4f58-8e66-620a5c1aa958'),
 ('finess', '84da3870-266c-4675-94dd-3ee037d668f5'),
 ('siren', '65838a1b-7c6e-4efa-9877-a0b9362e0ced'),
 ('finess', 'f8cf741f-1a4c-43b2-8b00-a9025db9cf53'),
 ('siren_epci', '678ca756-0d8a-45a2-bf3c-57f3ccb47b78'),
 ('nofinesset', '6d03f6e7-0936-4328-ae11-c266be8944b5'),
 ('nofinessej', '6d03f6e7-0936-4328-ae11-c266be8944b5'),
 ('SIREN', '46acb713-6d22-46b3-adcc-10cb93ce1c7e'),
 ('FINESS_ET', 'd0075329-2e20-4b0b-a3e9-f8c182e8d815'),
 ('FINESS_EJ', 'd0075329-2e20-4b0b-a3e9-f8c182e8d815'),
 ('SIREN', '6b47c253-81a8-4464-ad69-cd78c7a157a6'),
 ('f

In [ ]:
list_rows = []
n_sampled_files = 100
np.random.seed(42) 
for type_, val_tups in iindex_types_cols:
    sample = np.random.choice(val_tups, n_sampled_files, replace=False)
    row_dict = {}
    for col_name, id_ in sample:
        row_dict["columns"] = col_name
        row_dict["sample"] = get_sample(results, id_=id_, header=col_name, n_sample=10)
        row_dict["human_detected"] = type_
        row_dict["csv_detected"] = type_
        
    